# svg_dataset

> End-to-end functions taking in centerline-stroke SVG's and outputting deltas in Stroke-3 format.

In [ ]:
#| default_exp stroke3.svg_dataset

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# |hide

In [ ]:
#| export


from singleline_dataset.stroke3.path_transforms import svg_to_strokes
from singleline_dataset.stroke3.stroke3 import strokes_to_deltas, rdp_strokes


def stroke_rdp_deltas(rescaled_strokes, epsilon=2.0):
    rdp_strokes = rdp_strokes(rescaled_strokes, epsilon)
    deltas = strokes_to_deltas(rdp_strokes)

    ## roundtrip / sanity check
    # _rdp_strokes = stroke3.deltas_to_strokes(deltas)
    # display_plot.plot_strokes(_rdp_strokes)

    return deltas, rescaled_strokes

In [ ]:
#| export
import os

import numpy as np

from singleline_dataset.stroke3.display_svg import render_deltas, render_strokes
from singleline_dataset.stroke3.display_plot import *
from singleline_dataset.stroke3.stroke3 import *
from singleline_dataset.stroke3.svg_files import enumerate_files
from singleline_dataset.stroke3.path_joining import merge_until, splice_until


def svgs_to_deltas(input_dir, output_dir=None, target_size=200, total_n=1000, min_n=3, epsilon=1.0, limit=None):
    if output_dir and not os.path.isdir(output_dir):
        os.makedirs(output_dir)

    dataset = []
    for i, fname in enumerate(enumerate_files(input_dir)):
        if limit and i > limit:
            break
        input_fname = os.path.join(input_dir, fname)

        try:
            rescaled_strokes = svg_to_strokes(input_fname, total_n=total_n, min_n=min_n)

            joined_strokes, _ = merge_until(rescaled_strokes, dist_threshold=10.0)
            spliced_strokes, _ = splice_until(joined_strokes, dist_threshold=30.0)

            print(f"{fname}: {len(rescaled_strokes)} strokes -> {len(joined_strokes)} joined -> {len(spliced_strokes)} spliced")

            deltas = stroke_rdp_deltas(spliced_strokes, epsilon=epsilon)
            dataset.append(deltas)

            # monitor number of points before/after applying RDP path simplification algorithm
            raw_points = np.vstack(rescaled_strokes).shape[0]
            rdp_points = deltas.shape[0]
            print(f"{input_fname} points: raw={raw_points}, rdp={rdp_points}")

            if output_dir:
                def new_suffix(fname, suffix):
                    return os.path.join(output_dir, fname.replace(".svg", suffix))
                
                plot_strokes(rescaled_strokes, fname=new_suffix(fname, '.0_strokes.png'))
                plot_strokes(joined_strokes, fname=new_suffix(fname, '.1_joined.png'))
                plot_strokes(spliced_strokes, fname=new_suffix(fname, '.2_spliced.png'))
                plot_strokes(deltas_to_strokes(deltas), fname=new_suffix(fname, '.3_deltas.png'))

                with open(new_suffix(fname, ".raw.svg"), "w", encoding="utf-8") as raw_out:
                    raw_dwg = render_strokes(rescaled_strokes, target_size=target_size)
                    raw_dwg.write(raw_out, pretty=True)
                    print(f"\twrote {raw_output_fname}")

                with open(new_suffix(fname, ".preproc.svg"), "w", encoding="utf-8") as preproc_out:
                    preproc_dwg = render_deltas(deltas, target_size=target_size)
                    preproc_dwg.save(preproc_output_fname)
                    print(f"\twrote {preproc_output_fname}")
        except Exception as e:
            print(f"error processing {input_fname}: {e}")
            raise e
    return np.array(dataset, dtype=object)

In [ ]:
!mkdir ../outputs

mkdir: ../outputs: File exists


In [ ]:
input_dir = '../data/svg/'
output_dir = '../outputs'

# debug: only run for the first 10 files
limit = 10

svgs_to_deltas(input_dir, output_dir, limit=limit)

Minimum distance: 0.2946925297391963
From 51_-1 (4 points)
To 52_0 (4 points)
[0] - len(curr_strokes) = 52, min_dist = 0.2946925297391963
Minimum distance: 0.4971782261692783
From 42_0 (5 points)
To 48_-1 (5 points)
[1] - len(curr_strokes) = 51, min_dist = 0.4971782261692783
Minimum distance: 0.5111259422916135
From 25_0 (13 points)
To 33_0 (13 points)
[2] - len(curr_strokes) = 50, min_dist = 0.5111259422916135
Minimum distance: 0.5224572989728425
From 29_0 (8 points)
To 39_-1 (8 points)
[3] - len(curr_strokes) = 49, min_dist = 0.5224572989728425
Minimum distance: 0.5393953678378925
From 2_-1 (68 points)
To 39_0 (68 points)
[4] - len(curr_strokes) = 48, min_dist = 0.5393953678378925
Minimum distance: 0.6142969310507422
From 9_0 (31 points)
To 37_0 (31 points)
[5] - len(curr_strokes) = 47, min_dist = 0.6142969310507422
Minimum distance: 0.6550592528202287
From 27_0 (9 points)
To 38_-1 (9 points)
[6] - len(curr_strokes) = 46, min_dist = 0.6550592528202287
Minimum distance: 0.679443022996

NameError: name 'closest_splice_pair' is not defined

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()